In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path

from matplotlib import pyplot as plt

from nerfstudio.utils.eval_utils import eval_setup
# from ns_extension.utils.grouping import GroupingClassifier

[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/17/25 14:12:00.204 13429] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


### Load configuration

In [6]:
# Path to the config for a trained model
model_dir = Path('/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features')
mesh_dir = model_dir.parent / 'mesh_exports'
load_config = model_dir / '2025-07-11_171420/config.yml'

# Load the model
config, pipeline, checkpoint_path, step = eval_setup(load_config)

# Get the mesh directory
# mesh_dir = pipeline.datamanager.config.data.parent / "mesh_exports"

# config, pipeline, checkpoint_path, step = eval_setup(load_config)

[14:12:42] Auto image downscale factor of 2                                                 ]8;id=532303;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=658910;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[14:12:50] use color only optimization with sigmoid activation                                         ]8;id=50064;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=370701;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

### Try original mesh

In [5]:
def features2vertex(points, features, mesh, k=5, sdf_trunc=0.03):
    """
    Map point cloud features to mesh vertices using KNN over a KDTree.

    Args:
        points: (N, 3) array of input point cloud
        features: (N, D) array of per-point features
        mesh: final cleaned mesh after culling (mesh_0)
        k: number of nearest neighbors to use for weighting
        sdf_trunc: truncation distance for SDF
    
    Returns:
        features_kNN: (M, D) array of per-vertex features
    """

    vertices = np.asarray(mesh.vertices)

    # Build tree
    tree = cKDTree(vertices)

    # Query nearest vertex for each point
    distances, indices = tree.query(points, k=k)  # shape: (N,)

    # Mask points where nearest vertex is within truncation distance
    # Use distance to closest vertex (distance[:, 0]) for truncation mask
    valid_mask = distances[:, 0] <= sdf_trunc

    if not np.any(valid_mask):
        # No points within truncation distance, return zeros
        return np.zeros((len(vertices), features.shape[1]))

    # Filter distances, indices, and features by valid points
    distances = distances[valid_mask]
    indices = indices[valid_mask]
    features = features[valid_mask]

    # Weighting with Gaussian kernel
    sigma = np.mean(distances)  # or set manually
    weights = np.exp(- (distances**2) / (2 * sigma**2))
    weights /= weights.sum(axis=1, keepdims=True)  # normalize

    # Aggregate features per vertex
    features_kNN = np.zeros((len(vertices), features.shape[1]))

    # Use a counts array to normalize contributions per vertex later
    vertex_weight_sum = np.zeros((len(vertices), 1))

    # Accumulate weighted features
    for i in range(k):
        vertex_indices = indices[:, i]
        weighted_feats = features * weights[:, i:i+1]

        # Accumulate weighted features
        np.add.at(features_kNN, vertex_indices, weighted_feats)

        # Accumulate weights for normalization
        np.add.at(vertex_weight_sum, vertex_indices, weights[:, i:i+1])

    # Normalize aggregated features by summed weights (avoid div by zero)
    nonzero_mask = vertex_weight_sum.squeeze() > 0
    features_kNN[nonzero_mask] /= vertex_weight_sum[nonzero_mask]

    return features_kNN

Try KDTree search

In [7]:
import numpy as np
from scipy.spatial import cKDTree
import pyvista as pv
import open3d as o3d

# Assume:
#   points: (N, 3) array of input point cloud
#   features: (N, D) array of per-point features
#   mesh: final cleaned mesh after culling (mesh_0)

# 1. Build KDTree on the original point cloud
sdf_trunc = 0.03
model = pipeline.model

# mesh = pv.read('Open3dTSDFfusion_mesh.ply')
mesh = o3d.io.read_triangle_mesh("Open3dTSDFfusion_mesh.ply")

# points = model.means.detach().cpu().numpy()

features_mesh = features2vertex(
    points=model.means.detach().cpu().numpy(),
    features=model.distill_features.detach().cpu().numpy(),
    mesh=mesh,
    k=5,
    sdf_trunc=0.03
)

# # Create a KD tree of mesh vertices (surface)
# mesh_tree = cKDTree(mesh.vertices)

# # Query nearest vertex for each point
# distances, indices = mesh_tree.query(points, k=1)  # shape: (N,)

# # Truncate points based on distance from surface
# dist_mask = distances <= sdf_trunc

# indices


(277426, 13)

In [58]:
import torch

decoded_mesh_features = model.decoder.per_gaussian_forward(torch.tensor(features_mesh).to(model.device).to(torch.float32))

# decoded_mesh_features['samclip'].shape
positive_queries = ['ground', 'dirt']
negative_queries = ['tree', 'leaf']

similarity_map = model.similarity_fx(
    features=decoded_mesh_features[model.main_features_name].unsqueeze(0).permute(2, 1, 0), 
    positive=positive_queries, 
    negative=negative_queries,
    method='pairwise'
).squeeze(-1).detach().cpu().numpy()

In [75]:
model.decoder

TwoLayerMLP(
  (hidden_conv): Conv2d(13, 64, kernel_size=(1, 1), stride=(1, 1))
  (feature_branch_dict): ModuleDict(
    (dinov2): Conv2d(64, 384, kernel_size=(1, 1), stride=(1, 1))
    (samclip): Conv2d(64, 768, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [86]:
model.text_encoder.compute_similarity

<bound method MaskCLIPExtractor.compute_similarity of MaskCLIPExtractor(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
            )
            (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): Mu

Load and plot

### Example of meshing

While there are multiple methods provided, Open3DTSDFFusion works best. 

In [5]:
from ns_extension.utils.mesh import Open3DTSDFFusion

mesh_dir = mesh_dir / "mesh_exports"

mesher = Open3DTSDFFusion(
    load_config, 
    depth_trunc=3.0, 
    depth_name="median_depth",
    output_dir=mesh_dir
)

mesher.main()

[18:12:55] Auto image downscale factor of 2                                                 ]8;id=770843;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=603069;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[18:13:28] use color only optimization with sigmoid activation                                         ]8;id=932275;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=670032;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

Processing frames:   0%|          | 0/441 [00:00<?, ?it/s]/tmp/ns-extension/ns_extension/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()
/tmp/ns-extension/ns_extension/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()
/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.

[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=28616


Finished computing mesh: 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/mesh_exports/mesh_exports/Open3dTSDFfusion.ply

### Plot mesh

In [2]:
import pyvista as pv

# Basic loading and plotting
def load_and_plot_ply(file_path):
    """
    Load a PLY mesh file and display it with basic visualization
    """
    # Load the PLY file
    mesh = pv.read(file_path)
    
    # Print basic information about the mesh
    print(f"Number of points: {mesh.n_points}")
    print(f"Number of cells: {mesh.n_cells}")
    print(f"Bounds: {mesh.bounds}")
    
    # Create a plotter and add the mesh
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, scalars='RGB', rgb=True)
    plotter.show_axes()
    plotter.show()

In [36]:
def property_to_color(property_values, colormap='viridis'):
    """Convert property values to colors for visualization"""
    import matplotlib.cm as cm
    import matplotlib.colors as mcolors
    
    # Normalize values to [0,1]
    normalized = (property_values - property_values.min()) / (property_values.max() - property_values.min())
    
    # Apply colormap
    cmap = cm.get_cmap(colormap)
    colors = cmap(normalized)[:, :3]  # Remove alpha channel
    
    return colors

In [49]:
similarity_cmap = property_to_color(similarity_map.detach().cpu().numpy())

/tmp/ipykernel_13429/1942133074.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(colormap)


In [59]:
# mesh = pv.read('Open3dTSDFfusion_mesh.ply')
mesh.point_data['features'] = similarity_map


In [60]:
# Create a plotter and add the mesh
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='features', rgb=False)
plotter.show_axes()
plotter.show()

# mesh.plot(scalars='RGB', rgb=True)

Widget(value='<iframe src="http://localhost:43415/index.html?ui=P_0x7f00fdf9e2c0_5&reconnect=auto" class="pyvi…

In [4]:
mesh_fn = './Open3dTSDFfusion_mesh.ply'
load_and_plot_ply(mesh_fn)

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:43415/index.html?ui=P_0x7f03e57a89a0_1&reconnect=auto" class="pyvi…